## Imports

In [28]:
import requests
import json
import os

from fake_useragent import UserAgent
from fake_headers import Headers
from tqdm.notebook import tqdm

## Constants

## Helper Functions

In [29]:
def read_file_as_string(filename):
    """Returns contents of textfile as string.
    
    Opens file with readonly permission, reads the 
    text and filters out tabs and newlines, and 
    returns the result string.
    """
    with(open(filename, 'r')) as txt:
        return txt.read().replace('\n', '').replace('\t', '')
    
def fake_user_agent_2():
    header = Headers(
        browser="chrome",  # Generate only Chrome UA
        os="mac",  # Generate ony Windows platform
        headers=True  # generate misc headers
    ).generate()
    return header['User-Agent']

    
def fake_user_agent():
    agent = UserAgent(fallback=DEFAULT_USER_AGENT)
    return agent.chrome

def save_data(data, filepath):
    with(open(filepath, 'w')) as outfile:
        json.dump(data, outfile)
        
def create_data_filename(component_type, f1_cat, f1_idx, f1_val, f2_cat, f2_idx, f2_val, page):
    # We want the file to look something like: 
    # [COMPONENT]_[TYPE]_[FILTER_CATEGORY]_[FILTER_VALUE].json 
    # This means something like capacitor_ceramic_capacitance_2.2e-12.json
    return f"{SAVE_FILE_PREFIX}_{component_type}_{f1_cat}_idx_{f1_idx}_{f1_val}_{f2_cat}_idx_{f2_idx}_{f2_val}_start_{page}.{SAVE_FILE_SUFFIX}"

## Constants

In [39]:
API_ENDPOINT = 'https://octopart.com/api/v4/internal'

CURRENT_WD = os.getcwd()
QUERY_FILENAME = "query_no_seller.txt"

TMP_DIR = f"{CURRENT_WD}/tmp/"
SAVE_DIR = f"{CURRENT_WD}/data/"
SAVE_FILE_PREFIX = "capacitors"
SAVE_FILE_SUFFIX = "json"

DEFAULT_USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
MAX_PAGE_OFFSET = 1000
MAX_RESULTS = 100

In [40]:
QUERY_STRING = read_file_as_string(f"{CURRENT_WD}/{QUERY_FILENAME}")

## Headers

In [41]:
from aluminum_electrolytic_categories import capacitance, voltage_rating_dc, voltage_rating_ac, voltage_rating

def get_cookies(px):
    # For some reason this cookie is what is required in order to authenticate that we are
    # indeed a genuine user and not trigger a captcha page.
    cookies = {
        '_px': px,
    }
    return cookies

def get_header(agent):
    header = {
        'user-agent': agent,
    }
    return header

def get_payload(start=0, limit=10, f1_idx=0, f2_idx=0, f3_idx=0):
    payload = {
        'operationName': 'PricesViewSearch',
        'variables': {
            'country': 'US',
            'currency': 'USD',
            'filters': {
                'category_id': [
                    '6331',
                ],
                'capacitance': [
                    capacitance[f1_idx],
                ],
                'voltagerating_ac_': [
                    voltage_rating_ac[f2_idx],
                ]            
            },
            'in_stock_only': False,
            'limit': limit,
            'start': start,
        },
        'query': QUERY_STRING,
    }
    return payload

# Main Logic

## Logic Helpers

In [42]:
def request(i, j, n, px): 
    header = get_header(DEFAULT_USER_AGENT)
    payload = get_payload(start=n, limit=MAX_RESULTS, f1_idx=i, f2_idx=j)
    cookies = get_cookies(px)
    response = requests.post(API_ENDPOINT, cookies=cookies, headers=header, json=payload)
    try: 
        return response.json(), len(response.json()['data']['search']['results'])
    except:
        if 'text/html' in response.headers['Content-Type']:
            print('Got an HTML page!', response.content)
            return {}, -1
        else:
            print("No results found!")
            return {}, 0

def collect(data, f1_idx, f1_value, f2_idx, f2_value, page):
    # First save the data to a file just so we're not screwed in case something goes wrong.
    filename = create_data_filename("aluminum_electrolytic", "capacitance", f1_idx, f1_value, "voltage_rating_ac", f2_idx, f2_value, page)
    filepath = f"{TMP_DIR}{filename}"
    save_data(data, filepath)
    
def save_all(name, collection):
    filename = f"{name}.json"
    filepath = f"{SAVE_DIR}{filename}"
    save_data(collection, filepath)

## Logic

In [43]:
def run(px, page_start_idx=0, f1_idx=0, f2_idx=0, f3_idx=0):
    f1_reached = False
    f2_reached = False
    page_start_reached = False
    
    for i in tqdm(range(0, len(capacitance))):
        if i < f1_idx and not f1_reached:
            continue
        f1_reached = True
        
        for j in tqdm(range(0, len(voltage_rating_ac))):
            if j < f2_idx and not f2_reached:
                continue
            f2_reached = True
            
            for n in tqdm(range(0, MAX_PAGE_OFFSET, 100)):
                if n < page_start_idx and not page_start_reached:
                    continue
                page_start_reached = True
                
                response, num_results = request(i, j, n, px)
                
                if num_results == 0:
                    break
                elif num_results == -1:
                    print(f"Filter 1 IDX: {i}, Filter 2 IDX {j}, Page start IDX: {n}")
                    return
                    
                # Collect the data
                collect(response, i, capacitance[i], j, voltage_rating_ac[j], n)
                
                if num_results < 100:
                    print("There were <100 results, and we've reached the end of the pagination.")
                    break

In [54]:
PX = '5ZmELEFS86sHRoQrVzB6P2c/0Z1SKucpGbHwc/CajZFAmqgrek11rUreqiY0gljhWupNWIBkR3DzaQsUphxNtA==:1000:58ovm4A6e7RTotvFz/t2tm4hyNM4Bkqlnr8nwza4MfP9nBtA5EWTZlw4pvUTZ6Fd4YQ2EDOgL9eUpqNlALN3axuQbZ8BD81TNvrppQQL5AM+JrGSc37FzbKydqIjs2LdhK+zEibyy7k8ulOiHs7olv5IMVffsDdmrIabKmf3FqLmW+48EjhjmaKgZf/yx3CPZRpaqb1FlotCoN9F8zN2iDOWE6pru7QJSupyLqUoL6ppZWHWocalOF7LujNnbyzzZggK4EV/E4BpowgPbxUXCQ=='

In [55]:
page_start=0 # Increments of 100
f1=79
f2=18

run(PX, page_start_idx=page_start, f1_idx=f1, f2_idx=f2)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

No results found!


  0%|          | 0/10 [00:00<?, ?it/s]

...

No results found!


  0%|          | 0/10 [00:00<?, ?it/s]

No results found!


  0%|          | 0/10 [00:00<?, ?it/s]

No results found!


# Post Processing

### Aggregate all files into single JSON

In [61]:
output = {}

In [62]:
first = True
total = 0

FILES_DIR = f"{CURRENT_WD}/tmp/"
OUTPUT_FILE_NAME = "aluminum_electrolytic_ac_voltage"

def aggregate():
    global first, output, total
    for filename in tqdm(os.listdir(FILES_DIR)):
        if "capacitor" in filename:
            with(open(f"{FILES_DIR}{filename}", 'r')) as file:
                data = json.load(file)
                total += len(data["data"]["search"]["results"])
                print(total)
                if first: 
                    output = dict(data)
                    first = False
                    continue
                output["data"]["search"]["results"] += data["data"]["search"]["results"]

aggregate()

  0%|          | 0/490 [00:00<?, ?it/s]

2
3
7
24
28
30
31
33
40
41
46
47
48
49
61
64
65
66
67
68
69
72
73
74
75
76
79
97
99
103
104
111
112
113
116
118
130
131
137
138
140
142
147
172
186
199
212
230
233
234
241
242
243
244
245
259
260
261
265
273
276
277
299
302
303
304
305
308
325
342
344
346
347
348
351
355
359
361
362
364
365
369
370
388
403
406
407
419
421
431
437
456
463
480
481
497
516
520
521
522
524
526
527
537
552
553
558
559
578
586
587
589
590
594
597
600
601
606
607
609
612
614
617
619
620
622
629
630
639
642
648
649
650
664
666
674
675
678
679
680
682
683
685
706
708
709
711
713
714
716
721
722
730
732
734
736
752
755
758
780
783
785
789
792
808
809
810
812
815
816
819
825
830
832
839
853
856
858
879
886
888
889
898
899
917
918
920
921
928
931
934
935
936
938
955
959
961
968
987
988
989
990
993
996
1019
1020
1027
1043
1045
1046
1051
1063
1064
1066
1073
1074
1078
1081
1082
1083
1093
1094
1095
1098
1100
1101
1117
1118
1119
1120
1121
1125
1126
1127
1128
1133
1134
1137
1149
1165
1190
1193
1194
1199
1200
1201
1207
1

In [63]:
print(total)

2295


In [64]:
save_all(OUTPUT_FILE_NAME, output)

In [65]:
with(open(f"{SAVE_DIR}{OUTPUT_FILE_NAME}.json", 'r')) as file:
    data = json.load(file)
    print(len(data["data"]["search"]["results"]))

2295


### Remove all duplicates from the different datasets

In [72]:
first = True
output = {}

seen = []
OUTPUT_FILES = ["aluminum_electrolytic_dc_voltage", "aluminum_electrolytic_voltage_rating", "aluminum_electrolytic_ac_voltage"]
OUTPUT_FILE_FINAL = "aluminum_electrolytic_all"

def aggregate():
    global first, output
    for filename in tqdm(OUTPUT_FILES):
        with(open(f"{SAVE_DIR}{filename}.json", 'r')) as file:
            data = json.load(file)

            for part in tqdm(data["data"]["search"]["results"]):
                if part["part"]["id"] not in seen:
                    seen.append(part["part"]["id"])
                    if not first:
                        output["data"]["search"]["results"].append(part)   
            if first: 
                output = dict(data)
                first = False
                continue

In [73]:
aggregate()
save_all(OUTPUT_FILE_FINAL, output)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/66661 [00:00<?, ?it/s]

  0%|          | 0/66655 [00:00<?, ?it/s]

  0%|          | 0/2295 [00:00<?, ?it/s]

In [74]:
with(open(f"{SAVE_DIR}{OUTPUT_FILE_FINAL}.json", 'r')) as file:
    data = json.load(file)
    print(len(data["data"]["search"]["results"]))
    

66703
